In [ ]:
'''This is LSTM right now. Will be updated soon!'''

import tensorflow as tf
import numpy as np

#for generating binary data
from random import shuffle

#for plotting
import matplotlib.pyplot as plt

In [ ]:
#########################
#### DATA GENERATION ####
#########################

In [ ]:
#generating 2^20 pieces of data to for training (note: this spans the sample space!)
train_input = ['{0:020b}'.format(i) for i in range(2**20)]
shuffle(train_input)
train_input = [map(int,i) for i in train_input]
ti  = []

for i in train_input:
    temp_list = []
    for j in i:
            temp_list.append([j])
    ti.append(np.array(temp_list))
train_input = ti

In [ ]:
#creating one-hot encoding vectors for output classes
train_output = []
 
for i in train_input:
    count = 0
    for j in i:
        if j[0] == 1:
            count+=1
    temp_list = ([0]*21)
    temp_list[count]=1
    train_output.append(temp_list)

In [ ]:
#splitting train and test data (train: 10,000; test: 2^20-10,000)
NUM_EXAMPLES = 10000
test_input = train_input[NUM_EXAMPLES:]
test_output = train_output[NUM_EXAMPLES:]
 
train_input = train_input[:NUM_EXAMPLES]
train_output = train_output[:NUM_EXAMPLES]

In [ ]:
########################
#### MODEL CREATION ####
########################

In [ ]:
#defining input and output variables
data = tf.placeholder("float", [None, 20, 1])
output = tf.placeholder("float", [None, 21])

In [ ]:
#defining one LSTM cell to pass to the rnn
num_hidden = 24
cell = tf.nn.rnn_cell.BasicLSTMCell(num_hidden, state_is_tuple=True)

In [ ]:
#defining the rNN model; val = unrolled model; tf abstracts backpropagation through time (model unrolling and backprop on that FFN)
val, state = tf.nn.dynamic_rnn(cell, data, dtype="float")

#note: dynamic_rnn uses a while loop to create a graph with 'x' number of steps, as opposed to fixed rnn

In [ ]:
#defining weights and biases of appropriate sizes for the cell state
weight = tf.Variable(tf.truncated_normal([num_hidden, int(output.get_shape()[1])]))
bias = tf.Variable(tf.constant(value=0.1, shape=[output.get_shape()[1]]))

In [ ]:
#'val' will be last layer of rNN (conditionally dependent on other layers, so no need to use these others for my cost function)

#dealing with shaping issue from tf rNN
val = tf.transpose(val, [1,0,2])
last = tf.gather(val, int(val.get_shape()[0]) - 1)

#utilizing softmax ativation for multiclass loss function
prediction = tf.nn.softmax(tf.matmul(last, weight) + bias)

In [ ]:
#defining loss function; clipping gradients to solve exploding gradient problem; note ReLU itself (as opposed to vanilla rNN) solves vanishing gradient problem
cross_entropy = -tf.reduce_sum(output * tf.log(tf.clip_by_value(prediction, 1e-10, 1.0)))

In [ ]:
#defining error on test set
mistakes = tf.not_equal(tf.argmax(output, 1), tf.argmax(prediction, 1)) #tensor of errors
error = tf.reduce_mean(tf.cast(mistakes, "float")) #mean of errors

#preparing optimization function
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(cross_entropy)

In [ ]:
#initializing all variables
init = tf.global_variables_initializer()

In [ ]:
#########################
#### MODEL EXECUTION ####
#########################

In [ ]:
#defining arrays used for plotting cost function's convergence
avg_set = []
epoch_set=[]

#for displaying error
error_interval = 100


#executing graph
with tf.Session() as sess:
    sess.run(init)
    batch_size = 1000
    num_batches = int(len(train_input)/batch_size)
    epoch = 5000
    for epoch in range(epoch):
        loc = 0
        for j in range(num_batches):
            inp, out = train_input[loc:loc + batch_size], train_output[loc: loc+batch_size]
            loc += batch_size            
            cost = sess.run(cross_entropy, {data: inp, output: out})
            sess.run(minimize, {data: inp, output: out})
        print("Epoch=", str(epoch))
        
        #saving costs each epoch for plotting
        epoch_set = np.append(epoch_set, epoch)
        avg_set = np.append(avg_set, cost)
        
        if (epoch % error_interval == 0):
            incorrect = sess.run(error,{data: test_input[:100], output: test_output[:100]})
            print("Epoch= ", epoch, "; error = ", incorrect)
            
    sess.close()

In [ ]:
#ploting cost function convergence as network trains
plt.plot(epoch_set, avg_set)
plt.show()